# Tutorial for HBMCMC code

The set of modules in acrg_hbmcmc allow a convenient way to perform 'hierachical Bayesian MCMC'. \
That is, we want to infer emissions of a partiular species (and the influence from the domain boundary) using a Markov Chain Monte Carlo algorithm. The 'hierarchy' is that this depends on 'hyperparameters', which are generally the uncertainties in the non-latent parameters involved in the estimation (i.e. not those we are trying to derive – the emissions – but others that are necessary, e.g. the measurement error). \
For a work-in-progress introuction to inverse modelling see: https://www.overleaf.com/project/5f8d6217aeca1900019a84ce 

The code within acrg_hbmcmc relies heavily on other code in the ACRG repository, mainly name.py in acrg_name. This code is generally used to read footprints, a priori emissions etc from the ACRG directory structure. \
Currently the MCMC estimation is completed using the pymc3 library (see https://docs.pymc.io/), a well-established statistical library.

# What the code does conceptually
Rather than taking you through the whole code, we can just take a brief overview of the stages taken in the code. The overarching aim of the code is to estimate the emissions of some species in space over a particular period of time using measurements of that species. We assume that over the inversion period (the period of time we run the code over) the emissions are constant at a particular location.\
We estimate emissions in a limited region and, as well as the emissions, we also need consider the contribution to the measurement from background air – termed the boundary conditions. We also estimate these in the inversion process. To make things easier, we reparameterise our emissions so that we estimate by how much they are scaled up or down from some a priori value. What this means is that we adjust some initial best guess by a factor, which we limit to positive values as we are dealing only with emissions (and not sinks). The reason for doing this is that it is easier to do the statistics on a model where the numbers inferred are similar (i.e. generally all around a scaling factor of 1 rather than spanning the orders of magnitude seen by emissions) and that we can do stuff like force zero emissions over the oceans (i.e. nomatter how much we scale zero in our 'best guess', it will always be zero). In addition, we also estimate the errors in addition to the measurement error in our inverse method, and the uncertainty in this is propagated through to the emissions estimates. \
The code roughly follows these steps:
1) Read the user defined inputs from a file (see next section)\
2) Reads in the required measurement data which will inform the inversion\
3) Read in the footprints, which map emissions to the measurements. For each grid cell of the disperion model output (usually NAME), there is the mole fraction contribution to the measurement assuming that the emissions are our 'best guess' or a priori emissions from the emissions file.\
4) Reduces the dimensionality of the emissions from the dispersion model resolution to a coarser resolution.\
5) Filter the data if needed (e.g. if you only want daytime measurements)
6) Place the data into appropriate matrices for passing to an MCMC algorithm.\
7) Pass the data to an MCMC routine to estimate the emissions, errors and boundary conditions. This will prepare the various statistical models chosen to model the inference.\
8) Put the output into a netcdf file

## Running the code

The easiest way to run the hbmcmc code is to copy the file hbmcmc_input_template.ini in acrg_hbmcmc/config/ to your run directory, and edit this code with your desired set up. \
After editing the config script, and in this case saving it as a file called 'hbmcmc_ch4_run.ini', you can run it from the command line (assuming that both are in the same directory) using
```
python run_hbmcmc.py -c hbmcmc_ch4_run.ini
```
The code generally explains the various inputs, but we will go through them in a bit more detail below:

**species** is a single string of the species you wish to do an inversion for, e.g. "CH4" or "CFC-11". As it says above, checkout out acrg_species_info.json for the various options currently available, or add your own if needed. \
**sites** is a list containing the sites you wish to use measurements from defined by their 3-letter code, e.g. ["MHD", "TAC"] for Mace Head and Tacolneston measurement sites. Again, as above, see acrg_site_info.json for options or add your own. \
**meas_period** is a list of the averaging you wish to apply to the measurements at the differnt sites. Often not much is gained by having many measurements made at really high frequency, especially as the footprints are rarely such high resolution. Instead we may wish to use a coarser frequency, e.g. every 6 hours, where all measurements made within a 6-hour period are averaged into a single measurement. A meas_period much be supplied in the list for each site, e.g. ["6H","6H"]. \
**start_date** is when you want the inversion to begin, as a string (see above). \
**end_date** is when you want the inversion to end, as a string (see above). Note that the day is not included (e.g. "2000-01-01" would be until 23:59 on 1999-12-31) \



These inputs provide information about where the measurement data is read from, and to use measurements that use something other than the default set-up. These inputs can probably be ignored if you are just getting started. \
**inlet** is the height at which the air is sampled from. If set to None, the default will be used. For many sites, there is only one inlet anyway and so it will just default to this one height (e.g. MHD at 10m). Other sites have multiple inlets, e.g. TAC has inlets at 50m, 100m and 185m (NB. these heights are above ground level). TAC will default to 185m, but we may wish to use a different height, e.g. 100m, and so can specify this here. Note that you then have to specify this in a list of the same length as the *sites* input. So for MHD and TAC, we could write inlet=[None, "100m"] to use the 100m inlet at TAC and the defauls for MHD. \
**intrument** specified which instrument made the measurement. Many gases are measured at a site using multiple instruments. If this is the case, the defaul should reflect the best choice. Sometimes you might want to override this, e.g. if the default instrument was down for a long period. Again, if specifying this for one site you must have an entry for each site, e.g. to use the CRDS instrument at MHD and default at TAC you could use intrument=["CRDS", None]. \
**obs_directory** specifies a directory other than the standard ACRG structure to read the measurement data from, specified as a string. The directory should have the structure /<site>/<obs_file> where the site and the obs file reflect what you would expect in the ACRG directory structure. This might be e.g. if you have some experimental data that is not suitable to be stored in the main ACRG obs file structure.

This section of inputs is named "INPUT.PRIORS". This is a little misleading as it doesn't have anything to do with the priors... \
What this section does contain is the information about the region (defined by ACRG's preexisting regions) in which the inversion will take place and information about the footprints (sensitivities) that will be used to map the emissions to the measurements. \
**domain** is the model domain in which the inference will take place. Examples of a domain is "EUROPE" or "SOUTHAFRICA". You will probably have a good idea of which domain you're trying to infer before you start.\
**fpheight** is the disperion-model equivalent of the inlet height. The reason that this may be different to the true inlet height is that the topography in models does not always capture true topography (e.g. a site in a valley may be underground in the model). This has to be input as a dictionary and the height as magl, e.g. fpheight={"MHD":None, "TAC":"100magl"}.
**emissions_name** specifies if you want to use a particular emissions file. An emissions file is is containst spatial information about the 'best guess' of what the emissions are in a particular domain (such as from an inventory). By default the inversion will use the file without a tag or named total and the most recent emissions file available. E.g., if you are running CH4 in EUROPE and the most recent emissions file is from 2010, it would default to something like ch4_EUROPE_2010.nc. But, if you want to use the emissions file ch4-oil-production_EUROPE_2010.nc, then set emissions_name="oil-production".
**fp_directory** just specifies the path to the footprints you want to use if not the defaults. The names and directory structure must mirror those in e.g. /shared/LPDM/fp_NAME/. You may need this if you are, e.g. experimenting with different footprints that are not suitable for sharing.
**flux_directory** specifies the path to the emissions file you wish to use, if not the default ACRG path. The structure must mirror that of /shared/LPDM/emissions/.

Basis functions are the computational representation of the emissions. Most simply, this could be thought of as the underlying grid resolution of the inversion, but could also be something such a different countries or spatially distinct sectors. For the boundaries, this is how the boundary is broken down (e.g. as 4 sides, as a gradient etc.)\
**bc_basis_case** is how the inversion interprets the boundaries. Currently (unless this is out of date now) it can only handle "NESW", which means that the influence from the boundary at each cardinal direction is inferred.\
**fp_basis_case** is the basis function representation of the emissions. For example, if using the file 16x16_EUROPE_2012.nc, fp_basis_case="16x16". Set to None (and quadtree_basis=True) to create the emissions basis functions on the fly.\
**quadtree_basis** set to True uses a quadtree algorithm to find a suitable basis function representation for emissions. Set to False if fp_basis_case is not None. The quadtree algorithm recursively divides the domain until the desired number of basis functions is reached, with the aim that regions with a higher contribution to the measurement (higher signal to noise) and more spatial variablity will have a finer spatial resolution than those that contribute little or are spatially uniform (e.g. far from the measurement site, oceans if not emitters, etc.) This is based on the a priori ("best guess") emissions multiplied by the average footprint for the inversion period, to give a mole fraction contributon at the dispersion-model resolution. Firstly, the domain is split into 4 new basis functions. The basis function that is most variable is then split into four new basis functions. Again, the basis function is split into 4 more until the desired number is achieved. There will always be a multiple of 3n+1 basis function.\
**nbasis** is the number of desired basis functions if using the quadtree algorithm. Note that if this is not a multiple of 3n+1 it will take th closest nubmer that works. There is no 'correct' number to use, but it will be harder to estimate a larger number of basis functions. However, too few may not represent reality well. This will require some experimentation on a case-by-case basis.\
**basis_directory** is the directory if not using the default ACRG location for the basis function. Again, this should mirror that of the ACRG file structure.



Currently this bit can't be changed, but in future there may be different options (e.g. trans-dimensional MCMC) all called from the same input script.

The explanation above is quite self explanatory in terms of what to input. If in doubt, then see https://docs.pymc.io/api/distributions/continuous.html.

These inputs control the time period over which some variables are estimated. For example, if the inversion is over one year but we want to estimate the boundary conditions each month (rather than just once over the whole period). 
**bc_freq** is the frequency at which to estimate the boundary conditions. The way to input this is quite clearly explained above. Note that, as we're scaling, setting bc_freq=None does not mean that the boundary condition will have the same value for the whole inversion period, but it means that the whole period will be scaled by the same factor.\
**sigma_freq** is as bc_freq, but for the model uncertainty estimated in the process.\
**sigma_per_site** set to True estimates the model-measurement uncertainty for each site, else if set to False it does one estimate for all sites. 


The above is quite self explanatory. If the terms don't make sense then it's maybe best to do a little reading on MCMC algorithms. Some things to note, the number of iterations stored will be **nit** minus **burn**. That is, **nit** is the total iterations used for sampling. **tune** are iterations before sampling takes place, and so in total each MCMC chain will do **tune** + **nit** iterations.

[MCMC.NCHAIN]
; Number of chains to run simultaneously. Must be >=2 to allow convergence to be checked.

nchain = 2


This is the number of chains that will be run (or how many independent MCMC estimates you will do). Only one chain will ever be stored, as one chain should contain all the info you need. If it doesn't then you have to up the number of iterations (and/or tune it better). The reason for running more than one chain is to try to understand whether your estimate is correct or not. In reality there is no way of knowing for certain that your esimate is correct, but we can tell if it wrong. In short, if we run an MCMC sampler twice (2 chains) and they give different estimates then we know we haven't run it for long enough as it hasn't converged. The more chains we run, the more sure we can be that it is giving the correct answer. 

This adds an additional error to the measurement error. If, for example, our measurement instrument makes a measurement every hour, and we average this into 24H measurements, then averagingerror=True will add the variability in the measurements in this 24H period to the instrumental measurement error. The rationale behind this is that if the measurements are fairly similar, we can probably smooth them over 24H and represent them well in our emissions models. If they are very variable, then smoothing them over this window is likely not going to be represented well and thus leads to a larger error.

**outputpath** is simply the path to where you want your output to be stored.\
**outputname** is the tag you want to use to identify your output. E.g., if outputname="v1", for the domain "EUROPE" and specied "CH4", with the run starting 2010-01-01, your output would be called "CH4_EUROPE_v1_2010-01-01.nc".

# Other ways to run the code
You may wish to run the code for many different dates using identical inputs and not want to create a new config file each time. To just change the dates, e.g. to run the config file hbmcmc_ch4_run.ini for dates 2012-01-01 to 2013-01-01, you can run
```
python run_hbmcmc.py 2012-01-01 2013-01-01 -c hbmcmc_ch4_run.ini
```
In practice, you will want to run the script as a batch job, i.e. send it off to run on an HPC rather than run it on the command line. Below is an example shell script to run the hbmcmc code on Blue Pebble (BP1),
```
#!/bin/sh
#PBS -l select=1:ncpus=4:mem=40GB
#PBS -l walltime=12:00:00

export OMP_NUM_THREADS=2
export OPENBLAS_NUM_THREADS=2

cd /<path>/<to>/<RunFolder>/

python run_hbmcmc.py 2012-01-01 2013-01-01 -c hbmcmc_ch4_run.ini
```
You can run this on BP1, assuming the file is called ```run_PBS_ch4_EUROPE.sh``` using
```
qsub run_PBS_ch4_EUROPE.sh
```
If you wish to run this for many dates at the same time, you can use what's called an array job. The script below will run the config file for 10 years, with inversions over 1 year starting at the start of each year, starting in 2010. That is, the inversion will be for all of 2010, 2011, 2012...2019.
```
#!/bin/sh
#PBS -l select=1:ncpus=4:mem=40GB
#PBS -l walltime=12:00:00
#PBS -J 1-10

export OMP_NUM_THREADS=2
export OPENBLAS_NUM_THREADS=2

cd /<path>/<to>/<RunFolder>/

Start='2010-01-01'
Offset=$(expr $PBS_ARRAY_INDEX - 1)
Start_Increment="$Offset years"
Increment="1 year"
Start_Date=$(date -I -d "$Start + $Start_Increment") || exit -1
End_Date=$(date -I -d "$Start_Date + $Increment") || exit -1

python run_hbmcmc.py $Start_Date $End_Date -c hbmcmc_ch4_run.ini
```
And again you would run this script using the qsub command.

# More detailed explanation of the script
Here we can go through the main script in a little more detail. We will only focus on the top-level function ```fixedbasisMCMC``` in hbmcmc.py. This script generally calls all the relevant bits.

First we read in the measurement data we want to use in our inversion. Many of the inputs will be familiar from the config script. We can denote the collection of measurements **y** for convenience.  
```
    data = getobs.get_obs(sites, species, start_date = start_date, end_date = end_date, 
                         average = meas_period, data_directory=obs_directory,
                          keep_missing=False,inlet=inlet, instrument=instrument, max_level=max_level)
```

Next we want to read in our sensitivities, or the mapping from emissions to measurements. Note, as mentioned in the beginning, we actually want the senstivities multiplied by the a priori emissions to give a contribution in terms of mole fraction which can be scaled up and down. Let's denote this mapping **H** when in matrix form, such that when multiplied by sclaing **x**, **y=Hx**.
```
    fp_all = name.footprints_data_merge(data, domain=domain, calc_bc=True, 
                                        height=fpheight, 
                                        fp_directory = fp_directory,
                                        bc_directory = bc_directory,
                                        flux_directory = flux_directory,
                                        emissions_name=emissions_name)
```

The next bit just makes sure that there is some measurement data and footprints available – if not then there's no point in carrying on
```
    for site in sites:
        for j in range(len(data[site])):
            if len(data[site][j].mf) == 0:
                print("No observations for %s to %s for %s" % (start_date, end_date, site))
    if sites[0] not in fp_all.keys():
        print("No footprints for %s to %s" % (start_date, end_date))
        return
    
    print('Running for %s to %s' % (start_date, end_date))
```

This bit of code was a bit of hack – in short, there are different ways to quantify measurement error. Here we just say that if it contains both error metrics, just use the one called 'variability' as we can't use both. This is also explained in the comment.
```
    #If site contains measurement errors given as repeatability and variability, 
    #use variability to replace missing repeatability values, then drop variability
    for site in sites:
        if "mf_variability" in fp_all[site] and "mf_repeatability" in fp_all[site]:
            fp_all[site]["mf_repeatability"][np.isnan(fp_all[site]["mf_repeatability"])] = \
                fp_all[site]["mf_variability"][np.logical_and(np.isfinite(fp_all[site]["mf_variability"]),np.isnan(fp_all[site]["mf_repeatability"]) )]
            fp_all[site] = fp_all[site].drop_vars("mf_variability")
```

This was explained in the config section, so I'll just copy and pase what was said there "This adds an additional error to the measurement error. If, for example, our measurement instrument makes a measurement every hour, and we average this into 24H measurements, then averagingerror=True will add the variability in the measurements in this 24H period to the instrumental measurement error. The rationale behind this is that if the measurements are fairly similar, we can probably smooth them over 24H and represent them well in our emissions models. If they are very variable, then smoothing them over this window is likely not going to be represented well and thus leads to a larger error."
```
    #Add measurement variability in averaging period to measurement error
    if averagingerror:
        fp_all = setup.addaveragingerror(fp_all, sites, species, start_date, end_date,
                                   meas_period, inlet=inlet, instrument=instrument,
                                   obs_directory=obs_directory)
```

Once again, this bit was explained fairly well in the config section, i.e. "Basis functions are the computational representation of the emissions. Most simply, this could be thought of as the underlying grid resolution of the inversion, but could also be something such a different countries or spatially distinct sectors." Here we compute the user's choice of whether to use a premade basis function representation or make one on the fly using a quadtree algorithm, explained as "The quadtree algorithm recursively divides the domain until the desired number of basis functions is reached, with the aim that regions with a higher contribution to the measurement (higher signal to noise) and more spatial variablity will have a finer spatial resolution than those that contribute little or are spatially uniform (e.g. far from the measurement site, oceans if not emitters, etc.) This is based on the a priori ("best guess") emissions multiplied by the average footprint for the inversion period, to give a mole fraction contributon at the dispersion-model resolution. Firstly, the domain is split into 4 new basis functions. The basis function that is most variable is then split into four new basis functions. Again, the basis function is split into 4 more until the desired number is achieved. There will always be a multiple of 3n+1 basis function." A basis function file will be temporarily saved in your run folder in a directory with some long name like Temp028ajfi.... and will be deleted at the end of the run (but also saved in the output). If for some reason the run fails after this point but before the end, then it will have to be removed manually.
```
    #Create basis function using quadtree algorithm if needed
    if quadtree_basis:
        if fp_basis_case != None:
            print("Basis case %s supplied but quadtree_basis set to True" % fp_basis_case)
            print("Assuming you want to use %s " % fp_basis_case)
        else:
            tempdir = basis.quadtreebasisfunction(emissions_name, fp_all, sites, 
                          start_date, domain, species, outputname,
                          nbasis=nbasis)
            fp_basis_case= "quadtree"+species+"-"+outputname
            basis_directory = tempdir
    else:
        basis_directory = basis_directory
```

These two lines reparameterise the sensitivities to emissions and the boundaries to the user's choice of basis functions
```
    fp_data = name.fp_sensitivity(fp_all, domain=domain, basis_case=fp_basis_case,basis_directory=basis_directory)
    fp_data = name.bc_sensitivity(fp_data, domain=domain,basis_case=bc_basis_case)
```

Sometimes you may wish to filter data (e.g. to remove local pollution event or only take measurements from certain times in the day.  The lines below do this if required
```
    #apply named filters to the data
    fp_data = name.filtering(fp_data, filters)
```

This is another hacky bit so that we have the domain saved into the ```fp_data``` dataset
```
    for si, site in enumerate(sites):     
        fp_data[site].attrs['Domain']=domain
```

The next few lines turns everything we have read in into nice matrices that can be used in a statistical model. Our aim is to use the model **y=Hx+e** to estimate **x**, the scaling of emissions/boundary conditions, and also estimate **e**, the model-measurement error distribution, using our known measurements **y** and mapping **H**.\
Ultimately, this creates a hierarchical probabilistic model\
$p(x,\sigma \mid y) \propto p(y\mid x)p(x\mid\sigma)p(\sigma)$\
where $\sigma$ is the standard deviation(s) of the model-measurement error in addition to the known measurement error. The 'hierarchy' comes from the fact that the inference of **x** relies on $\sigma$.\
It's easier to split our matrix **H** into two separate matrices, one for the emissions and one for the boundary condtions, and this is what we do below.
```
#Get inputs ready
error = np.zeros(0)
Hbc = np.zeros(0)
Hx = np.zeros(0)
Y = np.zeros(0)
siteindicator = np.zeros(0)
for si, site in enumerate(sites):
    if 'mf_repeatability' in fp_data[site]:           
        error = np.concatenate((error, fp_data[site].mf_repeatability.values))
    if 'mf_variability' in fp_data[site]:
        error = np.concatenate((error, fp_data[site].mf_variability.values))

    Y = np.concatenate((Y,fp_data[site].mf.values)) 
    siteindicator = np.concatenate((siteindicator, np.ones_like(fp_data[site].mf.values)*si))
    if si == 0:
        Ytime=fp_data[site].time.values
    else:
        Ytime = np.concatenate((Ytime,fp_data[site].time.values ))

    if bc_freq == "monthly":
        Hmbc = setup.monthly_bcs(start_date, end_date, site, fp_data)
    elif bc_freq == None:
        Hmbc = fp_data[site].H_bc.values
    else:
        Hmbc = setup.create_bc_sensitivity(start_date, end_date, site, fp_data, bc_freq)

    if si == 0:
        Hbc = np.copy(Hmbc) #fp_data[site].H_bc.values 
        Hx = fp_data[site].H.values
    else:
        Hbc = np.hstack((Hbc, Hmbc))
        Hx = np.hstack((Hx, fp_data[site].H.values))
```


This line is just to help set up the $\sigma$ term so that they can be distributed in time and/or by site if needed (see explanation in the config file).
```
sigma_freq_index = setup.sigma_freq_indicies(Ytime, sigma_freq)
```

This is where the magic happens. Everything we have read in so far, an our description of the probabilistic mode, are passed to Pymc3 - a statistical library that perform MCMC to estimate what we wish to estimate. The emissions and boundary conditions are inferred using a method called NUTS (https://www.jmlr.org/papers/volume15/hoffman14a/hoffman14a.pdf) which is really efficient. The hyperparameters (i.e. model error) uses a Slice sampler (DOI: 10.1214/aos/1056562461). The reason for using a different sampler was originally because a slice sampler is faster, and in general you don't need as many iterations for the hyperparameter (unless you have lots of them) and so I just used a less efficient but faster one.\
The inputs to the ```fixedbasisMCMC``` function takes the three arguments
```
xprior={"pdf":"lognormal", "mu":1, "sd":1},
bcprior={"pdf":"lognormal", "mu":0.004, "sd":0.02},
sigprior={"pdf":"uniform", "lower":0.5, "upper":3},
```
These are the prior probabilities for the emissions, boundary conditions and hyperparameters. The function ```inferpymc3``` in inversion_pymc3.py gives a bit more detail on these, as done the description in the config file, but in short the first bit is the name of the PDF (as per pymc3) and then follow the parameters that describe that distribution. Note that **these are not one size fits all distributions**. These are just some value I was using at the time of putting this together, i.e. for some VSLS in South Africa. There are many to choose from and you should really spend some time thinking about what best described your case. E.g. a lognormal(1,1) distribution is fairly uninformative, i.e. there is a 95% probability that the true value is between 0.38-19.3 times your a priori distribution. Emissions can only be positive, with the most probably value being that of the emissions value. The mean, however, is ~4.5x the a priori value, so be careful with your metrics. Other positively bounded distributions could be better suited, such as the truncated normal, but again this distribution has sharp changes which may be unrealistic and/or cause problems when sampling. The point is, don't just randomly pick something - think about it and think how this may affect the metrics you might be quoting in a piece of work, e.g. if using a LN(1,1) and your data provides no info, and you report on the posterior mean, then of course your posterior mean will be larger than your a priori emissions.
```
#Run Pymc3 inversion
xouts, bcouts, sigouts, Ytrace, convergence, step1, step2 = mcmc.inferpymc3(Hx, Hbc, Y, error, siteindicator, sigma_freq_index,
       xprior,bcprior, sigprior,nit, burn, tune, nchain, sigma_per_site, verbose=verbose)
```

This next part takes the output from the MCMC code and makes some nice metrics, such as country totals, and saves everything to a netcdf file (see config description). Hopefully the variables names are all quite self-explanatory, and there should be everything you need in there to reproduce the results. You can plot the data and things like that using ```hbmcmc_post_process.py```. 
```
#Process and save inversion output
mcmc.inferpymc3_postprocessouts(xouts,bcouts, sigouts, convergence, 
                       Hx, Hbc, Y, error, Ytrace,
                       step1, step2, 
                       xprior, bcprior, sigprior,Ytime, siteindicator, sigma_freq_index, fp_data,
                       emissions_name, domain, species, sites,
                       start_date, end_date, outputname, outputpath,
                       basis_directory, country_file, country_unit_prefix,flux_directory)
```

This bit just remove the temporary directory containing the quadtree basis functions if used.
```
if quadtree_basis is True:
    # remove the temporary basis function directory
    shutil.rmtree(tempdir)
```

Finally, if you see this then it should mean that it ran through successfully without errors (although this doesn't necessarily mean that the results are any good!).
```
print("All done")
```